In [41]:
from pymongo import MongoClient
import pprint

import pandas as pd
import matplotlib.pyplot as plt

import requests

from bs4 import BeautifulSoup

import json
import datetime
import calendar
import time

In [42]:
#  Connect to the hosted MongoDB instance
client = MongoClient()

In [43]:
# Access already existing database
db = client['capstone1_bball_reference']

In [44]:
# Access already existing collection in database
all_seasons = db['all_seasons']

In [54]:
actual_links = db.all_seasons.distinct("link")

## Get HTML pages from MongoDB and parse with Beautiful Soup

In [79]:
def parse_to_df(links, starting_season,ending_season):
    
    date = []
    visiting = []
    visiting_score = []
    home = []
    home_score = []
    season = []
    notes = []
    
    for i,link in enumerate(links):
        
        if starting_season <= int(link.split("_")[1]) <= ending_season:
        
            sub_season = all_seasons.find_one({'link':link})['html']
            soup = BeautifulSoup(sub_season,'html.parser')

            for i,game in enumerate(soup.find_all('tr')):
                if game.a != None: #skips the chart label rows
                    datestring = game.find('a').text[5:]
                    dt = datetime.datetime.strptime(datestring,'%b %d, %Y') #change datestring to datetime object
                    date.append(dt)
                    visiting.append(game.find_all('td','left')[0].text)
                    home.append(game.find_all('td','left')[1].text)
                    season.append(soup.find_all('div','inactive')[0].find('li','index').text)
                    notes.append(game.find_all('td','left')[-1].text)
                    if len(soup.find_all('col'))==10: #takes care of the different column counts 
                        if (game.find_all('td','right')[1].text) != '':
                            visiting_score.append(int(game.find_all('td','right')[1].text))
                            home_score.append(int(game.find_all('td','right')[2].text))
                        else: 
                            visiting_score.append(0)
                            home_score.append(0)
                    else:
                        if (game.find_all('td','right')[0].text) != '':
                            visiting_score.append(int(game.find_all('td','right')[0].text))
                            home_score.append(int(game.find_all('td','right')[1].text))
                        else: 
                            visiting_score.append(0)
                            home_score.append(0)
                
    df = pd.DataFrame({'date':date,'season':season,'visiting_team':visiting,'visiting_score':visiting_score,
                  'home_team':home, 'home_score':home_score, 'notes':notes})
    return df

In [80]:
parse_to_df(actual_links, 2010, 2019)

,date,season,visiting_team,visiting_score,home_team,home_score,notes
0,2010-04-01,2009-10 NBA Season,Orlando Magic,97,Dallas Mavericks,82,
1,2010-04-01,2009-10 NBA Season,Portland Trail Blazers,92,Denver Nuggets,109,
2,2010-04-02,2009-10 NBA Season,Milwaukee Bucks,86,Charlotte Bobcats,87,
3,2010-04-02,2009-10 NBA Season,Miami Heat,105,Indiana Pacers,96,
4,2010-04-02,2009-10 NBA Season,Chicago Bulls,95,Washington Wizards,87,
...,...,...,...,...,...,...,...
12892,2018-09-29,2018-19 NBA Season,Portland Trail Blazers,0,Toronto Raptors,0,
12893,2018-09-30,2018-19 NBA Season,Charlotte Hornets,0,Boston Celtics,0,
12894,2018-09-30,2018-19 NBA Season,New Orleans Pelicans,0,Chicago Bulls,0,
12895,2018-09-30,2018-19 NBA Season,Denver Nuggets,0,Los Angeles Lakers,0,
